In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/home/asmolka/repos/exess/smolkaa.github.io/api/exospheres"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.landing_position

```
[1] landing_position(x0::AbstractVector, v0::AbstractVector; kwargs...)
[2] landing_position(x0::AbstractPosition, v0::AbstractVelocity; kwargs...)
```

Calculates the landing position of a particle starting at position `x0` (global spherical coordinates: radius, longitude, latitude), with initial velocity `v0` (local cartesian coordinates: x (east), y (north), z (up)), flying on a ballistic trajectory, i.e., only influenced by graviational forces.

Returns the landing position in global spherical coordinate as `r, lon, lat` ([1]) or as `GlobalSphericalPosition(r, lon, lat)` ([2]).

**Key-Word Arguments**

| Field |        Value | Unit | Description              |
|:----- | ------------:|:---- |:------------------------ |
| `m`   | `LUNAR_MASS` | [kg] | mass of planetary object |

**Notes**

  * calling the function with `<:Integer` arguments will promote them to `Float64`

**References**

  * N. Schörghofer: "USER GUIDE: Planetary-Code-Collection: Thermal and Ice Evolution Models for Planetary Surfaces", https://github.com/nschorgh/Planetary-Code-Collection
  * B. J. Butler, “The migration of volatiles on the surfaces of Mercury and the Moon,” Journal of Geophysical Research, vol. 102, no. E8, pp. 19,283–19,291, Aug. 1997, doi: 10.1029/97JE01347.


In [3]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.trajectory

```
[1] trajectory(x0::AbstractVector, v0::AbstractVector, ddx::Function; kwargs...)
[2] trajectory(x0::AbstractPosition, v0::AbstractVelocity, ddx::Function; kwargs...)
```

Calculate the trajectory of a particle starting at position `x0` (global cartesian coordinates), with initial velocity `v0` (global cartesian coordinates), given the  acceleration function `ddx` (global cartesian coordinates).

Returns a `ODESolution` object as the trajectory.

**Key-Word Arguments**

| Field    |          Value | Unit       | Description                            |
|:-------- | --------------:|:---------- |:-------------------------------------- |
| `alg`    |      `Tsit5()` |            | numerical solver algorithm             |
| `rmin`   | `LUNAR_RADIUS` | [m]        | minimum radius of computational domain |
| `rmax`   |          `1e9` | [m]        | maximum radius of computational domain |
| `tspan`  |   `(0f0,1f10)` | ([s], [s]) | time span of the integration           |
| `kwargs` |                |            | additional key-word arguments          |

The integration terminates if either the minimum or maximum radius is exceeded or if the end of the time span is reached.

**Notes**

  * calling the function with `<:Integer` arguments will promote them to `Float64`


In [4]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.ddx_lunar_gravity

```
[1] ddx_lunar_gravity(x::AbstractVector, [args...])
```

Acceleration function for the lunar gravity.
